# AWS RDS - CPU
This notebook shows the CPU on an AWS RDS: 

This early version of a notebook brings the data as is, without any analysis of the information. 
## Cells
- Configuration - open an AWS session to CloudWatch. 
- Get a lits of RDS servers
- Show the CPU (min, max, avg) of a given RDS, between two dates. 

## Resources
- https://www.metisdata.io/blog/hold-your-horses-postgres-how-to-debug-high-cpu-usage

## Configuration
- Configure the AWS credentials: access_key_id, secret_access_key, region_name
- Configure the PG connections string

In [11]:
import configparser

# Read from the Config file
try: 
    config = configparser.ConfigParser() 
    config.read_file(open(r'../ipynb.cfg'))
except Exception as e:
    print(f"Error opening the configuration file: {e}")

try: 
    # con_str = config.get('con_str', 'PG_AIRBASES')
    access_key_id = config.get('credentials', 'ACCESS_KEY_ID') 
    secret_access_key = config.get('credentials', 'SECRET_ACCESS_KEY')   
    region_name =  'eu-central-1' 
except Exception as e:
    print(f"Error opening the configuration file: {e}")



## Prereq
- pip install boto3
- pip install dash

## Show the CPU 
Desc: When reading the CPU history, you might want to see the data every 10 minutes. However, the notebook shows the data every 10 min from now. So if it runs at 17:58, CloudWatch returns the data at 17:58, 17:48, 17:38... That's why the function starts from a rounded time. 

TODO: Calculate the actual CPU, memory and IO from the instance type (such as  db.m6g.large)

In [12]:
import boto3
import pandas as pd
from datetime import datetime, timedelta

def create_aws_session(access_key_id, secret_access_key, region_name):
    try:
        # Create a session using AWS credentials
        session = boto3.Session(
            aws_access_key_id=access_key_id,
            aws_secret_access_key=secret_access_key,
            region_name=region_name
        )
        return session
    except Exception as e:
        print("An error occurred while creating the AWS session:", str(e))
        return None

# Function to round a datetime object to the nearest 10 minutes
def round_to_nearest_10_minutes(dt):
    minute = (dt.minute // 10) * 10
    return dt.replace(second=0, microsecond=0, minute=minute)

# Shows the CPU utilization of an RDS instance
def rds_cpu(rds_instance_id, session, period=10):
    try:
        metric_name = 'CPUUtilization'

        # Create a CloudWatch client using the existing AWS session
        cloudwatch = session.client('cloudwatch')

        # Calculate the end time (now) and round it to the nearest 10 minutes
        end_time = round_to_nearest_10_minutes(datetime.utcnow())

        # Calculate the start time as 7 days ago from the end time
        start_time = end_time - timedelta(days=1)

        # Convert start_time and end_time to ISO format
        start_time_iso = start_time.isoformat()
        end_time_iso = end_time.isoformat()

        # Get CPU utilization metric data (average, minimum, and maximum)
        response = cloudwatch.get_metric_data(
            MetricDataQueries=[
                {
                    'Id': 'm1',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Average',
                    },
                    'ReturnData': True,
                },
                {
                    'Id': 'm2',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Minimum',
                    },
                    'ReturnData': True,
                },
                {
                    'Id': 'm3',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Maximum',
                    },
                    'ReturnData': True,
                },
            ],
            StartTime=start_time_iso,
            EndTime=end_time_iso,
        )

        # Extract and return the CPU utilization data as a Pandas DataFrame
        avg_values = response['MetricDataResults'][0]['Values']
        min_values = response['MetricDataResults'][1]['Values']
        max_values = response['MetricDataResults'][2]['Values']
        timestamps = response['MetricDataResults'][0]['Timestamps']

        cpu_data = pd.DataFrame({
            'Timestamp': timestamps,
            'Average_CPUUtilization': avg_values,
            'Minimum_CPUUtilization': min_values,
            'Maximum_CPUUtilization': max_values
        })

        return cpu_data

    except Exception as e:
        print("An error occurred while retrieving CPU utilization:", str(e))
        return None

# Replace these values with your AWS credentials and RDS instance name
rds_instance_id = 'database-2'
session = create_aws_session(access_key_id, secret_access_key, region_name)

# Get the CPU utilization data for the specified RDS instance using the existing session
cpu_data = rds_cpu(rds_instance_id, session, period=600)  # Use a different period (e.g., 600) if needed

# Print the CPU utilization data
if cpu_data is not None:
     print(cpu_data)


                    Timestamp  Average_CPUUtilization  Minimum_CPUUtilization  \
0   2023-11-12 13:50:00+00:00               54.182500               42.233333   
1   2023-11-12 13:40:00+00:00               60.260833               44.408333   
2   2023-11-12 13:30:00+00:00               62.112500               52.483333   
3   2023-11-12 13:20:00+00:00               60.560833               53.158333   
4   2023-11-12 13:10:00+00:00               49.960000               36.158333   
..                        ...                     ...                     ...   
139 2023-11-11 14:40:00+00:00               35.764167               32.133333   
140 2023-11-11 14:30:00+00:00               34.958333               29.916667   
141 2023-11-11 14:20:00+00:00               49.392500               28.916667   
142 2023-11-11 14:10:00+00:00               46.438333               36.116667   
143 2023-11-11 14:00:00+00:00               42.552500               31.241667   

     Maximum_CPUUtilization

In [13]:
# Updated rds_cpu function
def rds_cpu(rds_instance_id, session, start_time, end_time=None, period=600):
    try:
        metric_name = 'CPUUtilization'

        # Create a CloudWatch client using the existing AWS session
        cloudwatch = session.client('cloudwatch')

        # If end_time is not provided, use the current time (now)
        if end_time is None:
            end_time = datetime.utcnow()

        # Round start_time and end_time to the nearest 10 minutes
        start_time = round_to_nearest_10_minutes(start_time)
        end_time = round_to_nearest_10_minutes(end_time)

        # Convert start_time and end_time to ISO format
        start_time_iso = start_time.isoformat()
        end_time_iso = end_time.isoformat()

        # Get CPU utilization metric data (average, minimum, and maximum)
        response = cloudwatch.get_metric_data(
            MetricDataQueries=[
                {
                    'Id': 'm1',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Average',
                    },
                    'ReturnData': True,
                },
                {
                    'Id': 'm2',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Minimum',
                    },
                    'ReturnData': True,
                },
                {
                    'Id': 'm3',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/RDS',
                            'MetricName': metric_name,
                            'Dimensions': [
                                {
                                    'Name': 'DBInstanceIdentifier',
                                    'Value': rds_instance_id
                                },
                            ]
                        },
                        'Period': period,
                        'Stat': 'Maximum',
                    },
                    'ReturnData': True,
                },
            ],
            StartTime=start_time_iso,
            EndTime=end_time_iso,
        )

        # Extract and return the CPU utilization data as a Pandas DataFrame
        avg_values = response['MetricDataResults'][0]['Values']
        min_values = response['MetricDataResults'][1]['Values']
        max_values = response['MetricDataResults'][2]['Values']
        timestamps = response['MetricDataResults'][0]['Timestamps']

        cpu_data = pd.DataFrame({
            'Timestamp': timestamps,
            'Average_CPUUtilization': avg_values,
            'Minimum_CPUUtilization': min_values,
            'Maximum_CPUUtilization': max_values
        })

        return cpu_data

    except Exception as e:
        print("An error occurred while retrieving CPU utilization:", str(e))
        return None


rds_instance_id = 'database-2'
start_time = datetime.utcnow() - timedelta(days=1)
end_time = datetime.utcnow()

# Get the CPU utilization data for the specified RDS instance using the existing session. 
# cpu_data is a DataFrame
cpu_data = rds_cpu(rds_instance_id, session, start_time, end_time, period=600)

# Convert start_time and end_time to formatted strings
start_time_str = start_time.strftime("%Y-%m-%d %H:%M:%S")
end_time_str = end_time.strftime("%Y-%m-%d %H:%M:%S")

# Concatenate rds_instance_id and formatted start/end times with the table title
table_title = f"RDS CPU Utilization Data - {rds_instance_id} (From: {start_time_str} To: {end_time_str})"

# Create a Plotly figure for the table
table_fig = go.Figure(data=[go.Table(
    header=dict(values=["Timestamp", "Average CPU Utilization", "Minimum CPU Utilization", "Maximum CPU Utilization"]),
    cells=dict(values=[cpu_data['Timestamp'], cpu_data['Average_CPUUtilization'], cpu_data['Minimum_CPUUtilization'], cpu_data['Maximum_CPUUtilization']])
)])

# Update the layout of the table figure
table_fig.update_layout(
    title=table_title,
    margin=dict(l=0, r=0, t=30, b=0)  # Adjust the margins as needed
)

# Display the table figure
table_fig.show()


In [14]:
import plotly.subplots as sp
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime, timedelta

# Assuming you have the cpu_data DataFrame containing the CPU utilization data
# Make sure the DataFrame is correctly populated from your previous code

# Create a subplot with multiple lines
fig = sp.make_subplots(rows=1, cols=1)  # You can adjust the number of rows and columns as needed

# Add traces for average, minimum, and maximum CPU utilization
trace_min = go.Scatter(x=cpu_data['Timestamp'], y=cpu_data['Minimum_CPUUtilization'], mode='lines', name='Minimum CPU Utilization')
trace_max = go.Scatter(x=cpu_data['Timestamp'], y=cpu_data['Maximum_CPUUtilization'], mode='lines', name='Maximum CPU Utilization')
trace_avg = go.Scatter(x=cpu_data['Timestamp'], y=cpu_data['Average_CPUUtilization'], mode='lines', name='Average CPU Utilization')

# Add the traces to the subplot

fig.add_trace(trace_min)
fig.add_trace(trace_max)
fig.add_trace(trace_avg)

# Customize the layout
fig.update_layout(
    title='RDS CPU Utilization Over Time',
    xaxis=dict(title='Timestamp'),
    yaxis=dict(title='CPU Utilization (%)'),
    legend=dict(title='Metric Type')
)

# Show the interactive chart
fig.show()
